# Test Results Review Report

This is supposed to help create the milestone report for now and eventuall generate it automatically. It is an interactive Jupyter document (or a static view of such a document) containing Julia code segments and their output. It is part of the JDP project which aims to create an *easily accessible* system for exploring test results and automating *arbitrary* workflows. Notebooks such as these are intended to provide an easy starting point for engineers and other technical users to create their own reports, possibly just by tweaking the existing ones.

Eventually this workbook should be easily installable locally or accessible from some remote location with zero knowledge of Julia or Jupyter. However right now it is not, but you can still try by visiting: https://github.com/richiejp/jdp.

Obviously you can also access the library from a REPL or use it in a traditional script or application, but Jupyter provides a nice, persistent, graphical environment. I won't discuss how to use Jupyter in this notebook (just click on help at the top), but will heavily annotate the code.

## Setup

First we need to load the JDP library which does the heavy lifting; importing and transforming the test result data from OpenQA into something useable. Note that this assumes you started this notebook by running `julia src/notebook.jl`.

> NOTE: It is required that you run this cell before the code cells following it. However not all of the cells need to be executed in order.

You may see a bunch of horrible angry red text when running this. Unfortunately this could either be info messages or error messages from the logging system, Jupyter treats both to a red background.

In [3]:
# Monitors library source files and recompiles them after most changes
import Revise

# Run the init script which will setup the JDP project if necessary
include("../src/init.jl")

# Bring DataFrame's _members_ into our namespace, so we can call them directly
using DataFrames

# Import some libraries from the JDP project
using JDP.Conf
using JDP.Trackers.OpenQA    # Contains functions for dealing with the OpenQA web API
using JDP.Trackers.Bugzilla  # Functions for accessing the Bugzilla API(s)
using JDP.Repository


┌ Info: Activating JDP package at /home/rich/julia/jdp/
└ @ Main /home/rich/julia/jdp/src/init.jl:5
┌ Info: Precompiling JDP [6d7e372e-c0bd-11e8-2b07-216b8359d694]
└ @ Base loading.jl:1186


> NOTE: Julia has a _very_ strong type system, but we can still assign variables like a dynamic language. For library code it is generally a good idea to explicitly state what types you are expecting, but in Notebook code we can just let the compiler guess the type.

Next we may download some new results for a given build or builds to our local cache. This usually takes a long time, hence why there is a local cache. The results are then loaded into a `DataFrame` object. Alternatively you can specify `Vector` as the second argument, in which case you will get an array of `TestResult` objects.

In [ ]:
# Get some job results from the openqa.suse.de (osd) OpenQA instance.
# Optional arguments (after the ';') like 'build' and 'groupid' are passed to the OpenQA API
df = Repository.fetch(OpenQA.TestResult, DataFrame, "osd"; refresh=false, groupid=116)

The function `describe` from the DataFrames package gives us some stats and information about the structure of the loaded data.

In [3]:
describe(df, stats = [:nunique, :min, :max, :eltype])

,variable,nunique,min,max,eltype
,Symbol,Int64,Any,Any,DataType
1,name,6401,1_autotest,zram03,String
2,suit,67,"[""LTP"", ""can""]","[""fstests"", ""xfs-xfs""]","Array{String,1}"
3,product,6,Unknown,sle-15-SP1-Installer-DVD,String
4,build,139,0250,98.1,String
5,result,6,canceled,softfailed,String
6,arch,4,aarch64,x86_64,String
7,refs,57,,,"Array{Ref,1}"


Look at the pretty table! We can also display graphs which could be even more delightful. Unfortunately it is slightly less pretty if you are viewing this as a static page.

## Failed tests for build

Let's look at what tests failed for a given build. First we need to filter out passed test results and results from other builds. Then we can group the results by test name and suit, amalgamating some of the columns to make the table easier to view. Filter is fairly simple, but the grouping is a bit more complex and there is a bit of Julia magic, see [Split-Apply-Combine](http://juliadata.github.io/DataFrames.jl/stable/man/split_apply_combine.html) for help.

> NOTE: Packages such as QUERY.jl allow one to use an SQL like syntax which is probably a lot easier to understand for most people.

In [4]:
build = "136.2"

# The syntax "var -> expr" is an anonymous function, strings starting with 'r' are regexs.
# In Julia you don't need to write 'return' (unless you want to return early), most 
# statements return whatever the value of the final expression is
fails = filter(r -> r.build == build && occursin(r"failed", r[:result]), df)

# group by name then apply the function defined by `do r ...` to each group
# Putting `do r` after `by` is like writing `by(r -> ...`. i.e. `do r` defines a function
# and passes it as the first argument to `by`.
fails_by_name = by(fails, [:name, :suit]) do r
    # 'by' first groups the results by name and suit then passes each group to us in the variable 'r'
    # we then use 'r' to produce a new DataFrame containing a single row. We return the new DataFrames 
    # and `by` then combines them... at least I think that is what happpens.
    DataFrame(
        # We have to write [] otherwise DataFrame creates a multi-row result (because r.result is an array)
        result = [unique(r.result)],
        arch = [unique(r.arch)],
        # Three dots `...` 'splats' an array (or tuple) into multiple function arguments 
        # and `vcat` concatenates it's arguments together
        refs = [filter(br -> br.tracker.api == nothing || br.tracker.api.name != "OpenQA", 
                          vcat(r.refs...)) |> unique]
        # also, and don't panic if this is a little more difficult to understand, 
        # 'unique' removes duplicate elements from a collection
    )
end

"$(nrow(fails_by_name)) tests failed this build"

"125 tests failed this build"

In [5]:
withenv("LINES" => 100) do
    display(fails_by_name)
end

,name,suit,result,arch,refs
,String,Array…,Array…,Array…,Array…
1,btrfs-078,"[""fstests"", ""btrfs""]","[""failed""]","[""x86_64"", ""aarch64""]",[bsc#1119089]
2,generic-263,"[""fstests"", ""xfs""]","[""failed""]","[""x86_64"", ""aarch64""]",[bsc#1119212]
3,xfs-266,"[""fstests"", ""xfs""]","[""failed""]","[""aarch64"", ""x86_64""]",[]
4,xfs-270,"[""fstests"", ""xfs""]","[""failed""]","[""aarch64"", ""x86_64""]",[]
5,xfs-278,"[""fstests"", ""xfs""]","[""failed""]","[""aarch64"", ""x86_64""]",[bsc#1110091]
6,xfs-281,"[""fstests"", ""xfs""]","[""failed""]","[""aarch64"", ""x86_64""]",[]
7,xfs-282,"[""fstests"", ""xfs""]","[""failed""]","[""aarch64"", ""x86_64""]",[]
8,xfs-283,"[""fstests"", ""xfs""]","[""failed""]","[""aarch64"", ""x86_64""]",[]
9,xfs-285,"[""fstests"", ""xfs""]","[""failed""]","[""aarch64"", ""x86_64""]",[bsc#1085914]


We probably have too many failures to display in Jupyter, so let's just try displaying failures for a subset of tests. We can focus on a particular test suit and remove tests which already appear to be tagged.

In [6]:
missing_bugrefs = filter(fails_by_name) do r
    length(r.refs) < 1 && # Remove tests which already have bug refs
    r.suit[1] == "LTP" &&    # Only include LTP results
    r.name != "boot_ltp" &&  # Don't include boot_ltp and shutdown_ltp modules
    r.name != "shutdown_ltp"
end

length(missing_bugrefs.name) > 0 ? missing_bugrefs : "Nothing to see here"

,name,suit,result,arch,refs
,String,Array…,Array…,Array…,Array…
1,mcast4-group-same-group,"[""LTP"", ""net_stress.multicast""]","[""failed""]","[""aarch64"", ""x86_64""]",[]
2,mcast4-group-source-filter,"[""LTP"", ""net_stress.multicast""]","[""failed""]","[""aarch64"", ""x86_64""]",[]
3,mcast6-group-same-group,"[""LTP"", ""net_stress.multicast""]","[""failed""]","[""aarch64"", ""x86_64""]",[]
4,mcast6-group-source-filter,"[""LTP"", ""net_stress.multicast""]","[""failed""]","[""aarch64"", ""x86_64""]",[]
5,clock_nanosleep02,"[""LTP"", ""syscalls""]","[""failed""]","[""aarch64""]",[]


In [7]:
has_bugrefs = filter(fails_by_name) do r
    !isempty(r.refs) && r.suit[1] == "fstests"
end

,name,suit,result,arch,refs
,String,Array…,Array…,Array…,Array…
1,btrfs-078,"[""fstests"", ""btrfs""]","[""failed""]","[""x86_64"", ""aarch64""]",[bsc#1119089]
2,generic-263,"[""fstests"", ""xfs""]","[""failed""]","[""x86_64"", ""aarch64""]",[bsc#1119212]
3,xfs-278,"[""fstests"", ""xfs""]","[""failed""]","[""aarch64"", ""x86_64""]",[bsc#1110091]
4,xfs-285,"[""fstests"", ""xfs""]","[""failed""]","[""aarch64"", ""x86_64""]",[bsc#1085914]
5,generic-050,"[""fstests"", ""xfs""]","[""failed""]","[""x86_64"", ""ppc64le"", ""aarch64""]",[bsc#1119210]
6,generic-091,"[""fstests"", ""xfs""]","[""failed""]","[""x86_64"", ""aarch64""]",[bsc#1119212]
7,btrfs-171,"[""fstests"", ""btrfs""]","[""failed""]","[""aarch64"", ""x86_64"", ""ppc64le""]",[bsc#1120036]
8,btrfs-172,"[""fstests"", ""btrfs""]","[""failed""]","[""aarch64"", ""x86_64""]",[bsc#1114270]
9,generic-471,"[""fstests"", ""btrfs""]","[""failed""]","[""aarch64"", ""x86_64""]",[bsc#1120171]


In [8]:
missing_bugrefs = filter(fails_by_name) do r
    length(r.refs) < 1 &&
    r.suit[1] == "fstests"
end

length(missing_bugrefs.name) > 0 ? missing_bugrefs : "Nothing to see here"

,name,suit,result,arch,refs
,String,Array…,Array…,Array…,Array…
1,xfs-266,"[""fstests"", ""xfs""]","[""failed""]","[""aarch64"", ""x86_64""]",[]
2,xfs-270,"[""fstests"", ""xfs""]","[""failed""]","[""aarch64"", ""x86_64""]",[]
3,xfs-281,"[""fstests"", ""xfs""]","[""failed""]","[""aarch64"", ""x86_64""]",[]
4,xfs-282,"[""fstests"", ""xfs""]","[""failed""]","[""aarch64"", ""x86_64""]",[]
5,xfs-283,"[""fstests"", ""xfs""]","[""failed""]","[""aarch64"", ""x86_64""]",[]
6,generic-521,"[""fstests"", ""btrfs""]","[""failed""]","[""aarch64""]",[]
7,btrfs-118,"[""fstests"", ""btrfs""]","[""failed""]","[""aarch64""]",[]
8,xfs-137,"[""fstests"", ""xfs""]","[""failed""]","[""aarch64"", ""x86_64""]",[]
9,xfs-142,"[""fstests"", ""xfs""]","[""failed""]","[""aarch64"", ""x86_64""]",[]


Let's try to find if any of these tests had bug refs in past builds. Note that the result of this depends on which of the above cells you ran last. This is because we reuse `missing_bugrefs`.

In [9]:
names_index = Set(missing_bugrefs.name) # Convert the name column into a hash set

past_results = by(filter(r -> r.name in names_index && length(r.refs) > 0, df), [:name, :suit]) do r
    DataFrame(refs = [filter(unique(vcat(r.refs...))) do br
                    br.tracker.api == nothing || br.tracker.api.name != "OpenQA"
                end])
end

,name,suit,refs
,String,Array…,Array…
1,generic-521,"[""fstests"", ""xfs""]",[bsc#1119212]
2,btrfs-139,"[""fstests"", ""btrfs""]",[bsc#1114393]
3,generic-263,"[""fstests"", ""xfs""]",[bsc#1119212]
4,xfs-491,"[""fstests"", ""xfs""]",[bsc#1114427]
5,xfs-492,"[""fstests"", ""xfs""]",[bsc#1114427]
6,xfs-493,"[""fstests"", ""xfs""]",[bsc#1114427]
7,generic-091,"[""fstests"", ""xfs""]",[bsc#1119212]


And now show the unique Bug references in a form you can click on.

> NOTE: DataFrame's `show` implementation for HTML deliberately calls `show` with the text/plain MIME type for its cell values. So we can not make the bug refs clickable while they are in the table.

In [10]:
unique(vcat(past_results.refs...))

[bsc#1119212](https://apibugzilla.suse.com/show_bug.cgi?id=1119212) [bsc#1114393](https://apibugzilla.suse.com/show_bug.cgi?id=1114393) [bsc#1114427](https://apibugzilla.suse.com/show_bug.cgi?id=1114427) 

Finally show tests for which we can not find any bug references

In [13]:
setdiff(names_index, past_results.name)

Set(["xfs-150", "xfs-145", "btrfs-152", "xfs-418", "btrfs-118", "xfs-146", "xfs-087", "xfs-154", "xfs-075", "xfs-158"  …  "xfs-156", "xfs-160", "xfs-281", "xfs-083", "xfs-085", "xfs-425", "xfs-429", "xfs-097", "xfs-159", "xfs-161", "xfs-263"])